In [74]:
%%writefile NeuralNetwork.py
print('imported Neural Network module')
import numpy as np
from numpy.linalg import eigvals

import random 
from InitializationVariables import *
from NodeFunctions import *

activationNodeCount = 7
sensorNodeCount = 10 + 2 * numRays 




def getANetwork(D_r):
    return generate_random_adjacency_matrix(D_r, 4, .8)

def generate_random_adjacency_matrix(N, avg_degree, spectral_radius):
    # Initialize adjacency matrix
    A = np.zeros((N, N))
    
    # Generate random edges to achieve the desired average degree
    num_edges = int(N * avg_degree/2)  # Undirected graph, so divide by 2
    edges = np.random.choice(N, size=(num_edges, 2), replace=True)
    
    # Generate random weights from a normal distribution centered around the spectral radius
    weights = np.random.normal(loc=1, scale=0.3, size=num_edges)
    weights = np.ones(num_edges)

    inhibitorProb = .2
    
    for idx in range(len(weights)):
        if random.random() < inhibitorProb:
            weights[idx] = -1
    
    # Populate the adjacency matrix with the generated edges and weights
    for idx, (i, j) in enumerate(edges):
        if i != j:  # Avoid self-loops
            A[i, j] +=weights[idx]
            A[j, i] +=weights[idx]  # Undirected graph

    current_radius = max(abs(eigvals(A)))
    A *= spectral_radius / current_radius

            
    return A



def getWin(D_r):
    N = sensorNodeCount
    M = D_r
    p = .2
    np.random.seed(41544)
    return np.random.choice([0, 1], size=(N, M), p=[1-p, p])


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def shouldFire(arr):

    arr = np.array(arr)
    input = softmax(arr)
    sigmoid = np.tanh(input)
    random_array = np.random.rand(*arr.shape)
    fire = random_array<sigmoid

    # print(sum(fire)) ##CDH
    return fire



def addMoves(move1, move2):
    move = [0, 0, 0]
    move[0] = move1[0] + move2[0]
    move[1] = move1[1] + move2[1]
    move[2] = move1[2] + move2[2]
    return move



def getAction(thought):


    fireNodes = shouldFire(thought)
    
    

    # print(f' Firing nodes: {fireNodes}')
    
    changeAngle = updateAngle(fireNodes[0]) - updateAngle(fireNodes[1])
    changeAngularVelocity = updateAngularVelocity(fireNodes[2]) - updateAngularVelocity(fireNodes[3])
    changeVelocity = updateVelocity(fireNodes[4]) - updateVelocity(fireNodes[5])


    

    movement = [changeAngle, changeAngularVelocity, changeVelocity] 
    
    killForward = functionKill(fireNodes[-1], killing)


    
    return movement, killForward, fireNodes


    
    
class NeuralNetwork:
    

    # W_in_network = []
    # A_network = []
    pVals = []

    age = 0
    angle = 0
    lastMove = [0, 0 ,0]


    sensorVals = []

    
    def think(self, theta, omega, xCoord, yCoord, barrierMask, killing, environment_pop_density, survivalMask):
        plotpoints = self.sense(theta, omega, xCoord, yCoord,killing, environment_pop_density, barrierMask, survivalMask)

        thought = self.applyMtx()
        
        movement, killForward,fireNodes = getAction(thought)


        if killing:
            print(f'{sum(fireNodes)}')
            
        return movement, killForward, plotpoints


    
    
    def sense(self, theta, omega, xCoord, yCoord,killing, environment_pop_density, barrierMask, survivalMask):
        self.age = self.age + 1
        
        sensorVals = [
            getLoc_X(xCoord),
            getLoc_Y(yCoord),
            getBoundary_Dist_x(xCoord),
            getBoundary_Dist_y(yCoord),
            getOscilatorVal(self.age),
            getAge(self.age),
            getRandom(),
            seeAngularVelocity(omega) 
        ]

        sensorVals.append(self.angle % (2*np.pi) / (2*np.pi) )
        sensorVals.append( (self.angle + np.pi) % (2*np.pi) / (2*np.pi) )

       
        plotpoints = []
        angles = np.linspace(-angleMax, angleMax, numRays)
        for angle in angles:
            results = rayDist(xCoord, yCoord, theta, angle, environment_pop_density, barrierMask, survivalMask) 

            distRatio = (vision_distance - results[0]) / vision_distance
            safetydistRatio = ( vision_distance - results[2]) / vision_distance
            
            sensorVals.append(distRatio)
            sensorVals.append(safetydistRatio)


            
            plotpoints += results[1]

       
        
        self.sensorVals = sensorVals
        return plotpoints


    def applyMtx(self):
        # self.sensorVals = np.random.rand(sensorNodeCount)
        
        # input = np.matmul(self.sensorVals, self.W_in_network)
        # idea = np.matmul(input, self.A_network)

        
        idea = np.matmul(self.sensorVals, W_A_Matmul)

        
        # print(f'pvals shape {self.pVals.shape}')
        # print(f'idea shape {idea.shape}')
        output = np.matmul(idea, self.pVals)
        return output
        
        
                     
    
    
    
    def __init__(self, gene):
        # self.W_in_network = W_In
        # self.A_network = A_network
        self.pVals = gene
        
        

W_In = getWin(D_r)
A_network = getANetwork(D_r)

W_A_Matmul = np.matmul(W_In, A_network)
    

Overwriting NeuralNetwork.py


In [23]:
def getStartGene():
    N = D_r
    M = activationNodeCount

    matrix = np.random.rand(N, M)
    
    # Normalize each Row so that its elements sum to 1
    matrix /= matrix.sum(axis=1, keepdims=True)
    return matrix

In [24]:
gene1 = getStartGene()

In [5]:
nn = NeuralNetwork(gene1)

gene = 0

W_in_network = []
A_network = []
idea = []
pVals = []


sensorVals = []

In [6]:
thought = nn.applyMtx()

In [7]:
go = np.tanh(thought)
random_array = np.random.rand(*thought.shape)

fire = random_array<go

In [8]:
getAction(fire, 2)

([0, 1, -1],
 False,
 array([ True,  True, False,  True,  True,  True, False,  True,  True,
         True,  True, False,  True,  True]))

In [11]:
generate_random_adjacency_matrix(10, 3, 1)
    

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.22354831, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.22354831,
        0.        , 0.        , 0.        , 0.        , 0.44709663],
       [0.        , 0.        , 0.        , 0.        , 0.44709663,
        0.44709663, 0.22354831, 0.44709663, 0.        , 0.        ],
       [0.        , 0.        , 0.22354831, 0.44709663, 0.        ,
        0.        , 0.        , 0.22354831, 0.        , 0.22354831],
       [0.        , 0.        , 0.        , 0.44709663, 0.        ,
        0.        , 0.        , 0.22354831, 0.        , 0.        ],
       [0.        , 0.22354831, 0.        , 0.22354831, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.       

In [12]:
row_sums = np.sum(generate_random_adjacency_matrix(5, 3, 1), axis=1)

np.mean(row_sums)

0.6954674622293305

In [13]:
a = np.array([1,3,0,0,0,1])

In [14]:
a-.5

array([ 0.5,  2.5, -0.5, -0.5, -0.5,  0.5])

In [26]:
gene1.T

array([[0.08728288, 0.13125765, 0.06479491, 0.14661581, 0.05313724,
        0.00915508, 0.05228094, 0.09187529, 0.12627819, 0.0304562 ],
       [0.09268235, 0.10536022, 0.11109632, 0.00439719, 0.06721675,
        0.09499961, 0.11387551, 0.07155698, 0.14693627, 0.12197629],
       [0.07230341, 0.03475811, 0.01914879, 0.08175738, 0.10306191,
        0.06945736, 0.05178177, 0.01533049, 0.05471   , 0.02453939],
       [0.12265959, 0.00689958, 0.10268143, 0.08740112, 0.06928212,
        0.06018746, 0.11501883, 0.10973553, 0.10819768, 0.09618899],
       [0.05547546, 0.00858275, 0.0927849 , 0.07398577, 0.08010209,
        0.00582216, 0.10552795, 0.00184208, 0.07492454, 0.10713017],
       [0.02474275, 0.07077355, 0.00488222, 0.13284228, 0.02426715,
        0.13457375, 0.12648015, 0.12580787, 0.030205  , 0.07756967],
       [0.11050394, 0.06299572, 0.09972981, 0.06320679, 0.13538266,
        0.08599282, 0.02557066, 0.15126969, 0.06850621, 0.02386549],
       [0.03785605, 0.00976454, 0.0340637

In [33]:
a = [True, False, True]
print(sum(a))

2


In [44]:
def shouldFire(arr):
    
    sigmoid = np.tanh(arr)
    random_array = np.random.rand(*arr.shape)
    fire = random_array<sigmoid
    
    return fire

In [48]:
a = np.linspace(2,0,20) -1
a

array([ 1.        ,  0.89473684,  0.78947368,  0.68421053,  0.57894737,
        0.47368421,  0.36842105,  0.26315789,  0.15789474,  0.05263158,
       -0.05263158, -0.15789474, -0.26315789, -0.36842105, -0.47368421,
       -0.57894737, -0.68421053, -0.78947368, -0.89473684, -1.        ])

In [58]:
sum(shouldFire(a))

[-0.76159416 -0.71372547 -0.65811078 -0.59424986 -0.52189993 -0.44117128
 -0.35260985 -0.25724684 -0.15659555 -0.05258303  0.05258303  0.15659555
  0.25724684  0.35260985  0.44117128  0.52189993  0.59424986  0.65811078
  0.71372547  0.76159416]


7

In [26]:
 4 % 3 * 2

2

In [15]:
import numpy as np

In [33]:
def shouldFire(arr):
    arr = np.array(arr)
    
    maxVal = max(arr.min(), arr.max(), key=abs)

    input = arr/maxVal

    
    sigmoid = np.tanh(input)
    print(sigmoid)
    random_array = np.random.rand(*arr.shape)
    fire = random_array<sigmoid
    
    return fire

In [38]:
arr = [9,1,3,5,3,-5]
shouldFire(arr)

[ 0.76159416  0.11065611  0.32151274  0.5046724   0.32151274 -0.5046724 ]


array([ True, False, False, False, False, False])